In [12]:
from Bio import SeqIO
from Bio import Entrez
import pandas as pd

### Obtener las secuencias en formato FASTA de las proteínas

In [13]:
# Configurar dirección de correo electrónico para acceder a Entrez
Entrez.email = "alegaviermoreno@gmail.com"

def obtener_secuencias_fasta(identificadores):
    # Obtener las secuencias FASTA de UniProt
    handle = Entrez.efetch(db="protein", id=identificadores, rettype="fasta", retmode="text")
    
    # Leer las secuencias en formato FASTA
    records = list(SeqIO.parse(handle, "fasta"))
    handle.close()

    return records

# Leer los identificadores desde un archivo (un identificador por línea)
with open("C:\\Users\\Ale\\Documents\\ProteinsIDs.txt", "r") as file:
    identificadores = [line.strip() for line in file]

# Obtener las secuencias FASTA
secuencias_fasta = obtener_secuencias_fasta(identificadores)

# Eliminar la proteína con identificador 'P22352'
secuencias_filtradas = [record for record in secuencias_fasta if 'P22352' not in record.id]

# Imprimir las secuencias en formato FASTA y guarda en un archivo
with open("sequence_proteins.fasta", "w") as fasta_file:
    for record in secuencias_fasta:
        # Imprime en consola
        print(f">{record.id}\n{record.seq}")
        
        # Guarda en archivo
        fasta_file.write(f">{record.id}\n{record.seq}\n")

>sp|A0A075B6J1.1|LV537_HUMAN
MAWTPLLLLLLSHCTGSLSQPVLTQPPSSSASPGESARLTCTLPSDINVGSYNIYWYQQKPGSPPRYLLYYYSDSDKGQGSGVPSRFSGSKDASANTGILLISGLQSEDEADYYCMIWPSNAS
>sp|A0A087WSX0.1|LV545_HUMAN
MAWTPLLLLFLSHCTGSLSQAVLTQPSSLSASPGASASLTCTLCSGINVGTYRIYWYQQKPGSPPQYLLRYKSDSDKQQGSGVPSRFSGSKDASANAGILLISGLQSEDEADYYCMIWHSSAS
>sp|A0A0A0MS15.1|HV349_HUMAN
MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGRSLRLSCTASGFTFGDYAMSWVRQAPGKGLEWVGFIRSKAYGGTTEYAASVKGRFTISRDDSKSIAYLQMNSLKTEDTAVYYCTR
>sp|A0A0A0MT89.2|KJ01_HUMAN
WTFGQGTKVEIK
>sp|A0A0B4J1V0.1|HV315_HUMAN
MEFGLSWIFLAAILKGVQCEVQLVESGGGLVKPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVKGRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTT
>sp|A0A0B4J1X5.1|HV374_HUMAN
MEFGLSWVFLVAILKGVQCEVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMHWVRQAPGKGLVWVSRINSDGSSTSYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAR
>sp|A0A0B4J1X8.1|HV343_HUMAN
MEFGLSWVFLVAILKGVQCEVQLVESGGVVVQPGGSLRLSCAASGFTFDDYTMHWVRQAPGKGLEWVSLISWDGGSTYYADSVKGRFTISRDNSKNSLYLQMNSLRTEDTALYYCAKD
>sp|A0A0B4J1Y9.1|HV372_HUMAN
MEFGLSWVFLVVILQGVQCEVQLVESGGGLV

### Obtener la localización subcelular de las proteínas con UniprotKB

In [14]:
import requests
import re

def obtener_informacion_uniprot(identificador):
    url = f'https://www.uniprot.org/uniprotkb/{identificador}.txt'
    response = requests.get(url)
    return response.text

def obtener_localizacion_subcelular(informacion_uniprot):
    # Utiliza expresiones regulares para encontrar la sección de localización subcelular
    match = re.search(r'-!- SUBCELLULAR LOCATION:(.*?)\n', informacion_uniprot, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

localizaciones = {}

for proteina in identificadores:
    informacion = obtener_informacion_uniprot(proteina)
    localizacion = obtener_localizacion_subcelular(informacion)
    localizaciones[proteina] = localizacion

# Crea un DataFrame de pandas
df_localizaciones = pd.DataFrame(list(localizaciones.items()), columns=['Proteina', 'Localizacion'])
df_localizaciones

,Proteina,Localizacion
0,A0A075B6J1,"Secreted {ECO:0000303|PubMed:20176268,"
1,A0A087WSX0,"Secreted {ECO:0000303|PubMed:20176268,"
2,A0A0A0MS15,"Secreted {ECO:0000303|PubMed:20176268,"
3,A0A0A0MT89,"Secreted {ECO:0000303|PubMed:20176268,"
4,A0A0B4J1V0,"Secreted {ECO:0000303|PubMed:20176268,"
...,...,...
236,Q9UKU9,Secreted.
237,Q9UKX5,Membrane; Single-pass type I membrane protein.
238,Q9UMD9,"Cell junction, hemidesmosome. Membrane; Single-"
239,Q9Y4K0,"Secreted, extracellular space, extracellular"


In [33]:
identificador_prueba = 'O15195'
informacion_prueba = obtener_informacion_uniprot(identificador_prueba)
print(informacion_prueba)

ID   VILL_HUMAN              Reviewed;         856 AA.
AC   O15195; A8MZP1; Q9BT80; Q9BWH7;
DT   20-JUN-2001, integrated into UniProtKB/Swiss-Prot.
DT   15-MAR-2005, sequence version 3.
DT   08-NOV-2023, entry version 180.
DE   RecName: Full=Villin-like protein;
GN   Name=VILL;
OS   Homo sapiens (Human).
OC   Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia;
OC   Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae;
OC   Homo.
OX   NCBI_TaxID=9606;
RN   [1]
RP   NUCLEOTIDE SEQUENCE [LARGE SCALE GENOMIC DNA].
RX   PubMed=16641997; DOI=10.1038/nature04728;
RA   Muzny D.M., Scherer S.E., Kaul R., Wang J., Yu J., Sudbrak R., Buhay C.J.,
RA   Chen R., Cree A., Ding Y., Dugan-Rocha S., Gill R., Gunaratne P.,
RA   Harris R.A., Hawes A.C., Hernandez J., Hodgson A.V., Hume J., Jackson A.,
RA   Khan Z.M., Kovar-Smith C., Lewis L.R., Lozado R.J., Metzker M.L.,
RA   Milosavljevic A., Miner G.R., Morgan M.B., Nazareth L.V., Scott G.,
RA   Sodergren E., 

Dividir las proteínas en secretadas e intracelulares.

In [17]:
# Filtrar las filas con localización 'Cell membrane'
df_cell_membrane = df_localizaciones[df_localizaciones['Localizacion'].str.contains('Cell membrane', case=False, na=False)]
# Filtrar las filas sin localización 'Secreted'
df_secreted = df_localizaciones[df_localizaciones['Localizacion'].str.contains('Secreted', case=False, na=False)]

In [18]:
df_secreted

,Proteina,Localizacion
0,A0A075B6J1,"Secreted {ECO:0000303|PubMed:20176268,"
1,A0A087WSX0,"Secreted {ECO:0000303|PubMed:20176268,"
2,A0A0A0MS15,"Secreted {ECO:0000303|PubMed:20176268,"
3,A0A0A0MT89,"Secreted {ECO:0000303|PubMed:20176268,"
4,A0A0B4J1V0,"Secreted {ECO:0000303|PubMed:20176268,"
...,...,...
223,Q9BXJ4,Secreted {ECO:0000305}.
228,Q9HC84,Secreted.
230,Q9NR99,"Secreted {ECO:0000269|PubMed:25326458,"
236,Q9UKU9,Secreted.


In [6]:
membrane = df_cell_membrane['Proteina']
membrane

16     O15427
18     O43653
59     P06703
60     P06731
62     P08134
63     P08174
67     P08637
74     P11166
80     P13688
81     P13987
85     P15313
89     P17342
113    P33241
141    P80723
154    Q13421
155    Q13443
159    Q14002
173    Q16553
183    Q5XXA6
186    Q6PIU2
203    Q8NFJ5
207    Q92743
231    Q9NRS4
233    Q9NZM1
234    Q9P121
Name: Proteina, dtype: object

In [7]:
membrane.to_csv('proteins_cellmembrane.txt', sep='\t', index=False)

In [8]:
# Configurar dirección de correo electrónico para acceder a Entrez
Entrez.email = "alegaviermoreno@gmail.com"

def obtener_secuencias_fasta(identificadores):
    # Obtener las secuencias FASTA de UniProt
    handle = Entrez.efetch(db="protein", id=identificadores, rettype="fasta", retmode="text")
    
    # Leer las secuencias en formato FASTA
    records = list(SeqIO.parse(handle, "fasta"))
    handle.close()

    return records

# Leer los identificadores desde un archivo (un identificador por línea)
with open("C:\\Users\\Ale\\Documents\\proteins_cellmembrane.txt", "r") as file:
    identificadores = [line.strip() for line in file]

# Obtener las secuencias FASTA
secuencias_fasta = obtener_secuencias_fasta(identificadores)

# Eliminar la proteína con identificador 'P22352'
secuencias_filtradas = [record for record in secuencias_fasta if 'P22352' not in record.id]

# Imprimir las secuencias en formato FASTA y guarda en un archivo
with open("proteins_cellmembrane.fasta", "w") as fasta_file:
    for record in secuencias_fasta:
        # Imprime en consola
        print(f">{record.id}\n{record.seq}")
        
        # Guarda en archivo
        fasta_file.write(f">{record.id}\n{record.seq}\n")

>sp|O15427.1|MOT4_HUMAN
MGGAVVDEGPTGVKAPDGGWGWAVLFGCFVITGFSYAFPKAVSVFFKELIQEFGIGYSDTAWISSILLAMLYGTGPLCSVCVNRFGCRPVMLVGGLFASLGMVAASFCRSIIQVYLTTGVITGLGLALNFQPSLIMLNRYFSKRRPMANGLAAAGSPVFLCALSPLGQLLQDRYGWRGGFLILGGLLLNCCVCAALMRPLVVTAQPGSGPPRPSRRLLDLSVFRDRGFVLYAVAASVMVLGLFVPPVFVVSYAKDLGVPDTKAAFLLTILGFIDIFARPAAGFVAGLGKVRPYSVYLFSFSMFFNGLADLAGSTAGDYGGLVVFCIFFGISYGMVGALQFEVLMAIVGTHKFSSAIGLVLLMEAVAVLVGPPSGGKLLDATHVYMYVFILAGAEVLTSSLILLLGNFFCIRKKPKEPQPEVAAAEEEKLHKPPADSGVDLREVEHFLKAEPEKNGEVVHTPETSV
>sp|O43653.2|PSCA_HUMAN
MAGLALQPGTALLCYSCKAQVSNEDCLQVENCTQLGEQCWTARIRAVGLLTVISKGCSLNCVDDSQDYYVGKKNITCCDTDLCNASGAHALQPAAAILALLPALGLLLWGPGQL
>sp|P06703.1|S10A6_HUMAN
MACPLDQAIGLLVAIFHKYSGREGDKHTLSKKELKELIQKELTIGSKLQDAEIARLMEDLDRNKDQEVNFQEYVTFLGALALIYNEALKG
>sp|P06731.4|CEAM5_HUMAN
MESPSAPPHRWCIPWQRLLLTASLLTFWNPPTTAKLTIESTPFNVAEGKEVLLLVHNLPQHLFGYSWYKGERVDGNRQIIGYVIGTQQATPGPAYSGREIIYPNASLLIQNIIQNDTGFYTLHVIKSDLVNEEATGQFRVYPELPKPSISSNNSKPVEDKDAVAFTCEPETQDATYLWWVNNQSLPVSPRLQLSNGNRTLTLFNVTRNDTASYKCETQNPVSARRSDSVI